1. Simulate noisy (uniform, multiplicative) measurements from model with "true" parameters
2. Run Levenberg-Marquardt least squares optimisation to find "best fit" parameters (note the mismatch between noise and cost function types)
3. Run MBAM to find out if reduction is similar (both original and log-space assays)

Rinse and repeat

In [1]:
# Helper functions

# Get a timestamp in useful format
timestamp()=Dates.format(now(),"_yyyymmddTHHMMSSss")

timestamp (generic function with 1 method)

### Simulate data, look at both assays and add noise in each separately (such that relative error is 10%)

In [2]:
# Simulate data points with noise
include("../BK_functions/bk_setup_script.jl")
ydata_noiseless = BK_simulator(phi0, hcat(x_grid...), model_id=0) # Create dataset
ydata_orig = ydata_noiseless.*(1+(2*rand(size(ydata_noiseless))-1)*0.1) # Add up to 10% noise (uniform, multiplicative)
ydata_log = log(ydata_noiseless).*(1+(2*rand(size(ydata_noiseless))-1)*0.1) # Add up to 10% noise (uniform, multiplicative, but in log space assay)

cur_time = timestamp();

## Fit noisy original assay and compute Hessian around best fit

In [3]:
# Optimise phi to fit the noisy data well
using LsqFit

# Store multiple fits (different initial point for LM, takes around 5 sec / fit)
all_fits = Array(Any, 24)
all_costs = Array(Any, length(all_fits))

for i1 = 1:length(all_fits)
    if i1==1
        phi_init = phi0;
    else
        phi_init = phi0.*(1+(2*rand(size(phi0))-1)*0.9) # Add up to 90% noise to initial params
    end
    all_fits[i1] = curve_fit((x,p) -> BK_simulator(exp(p), x), hcat(x_grid...), ydata_orig, log(phi_init));
    all_costs[i1] = sum(all_fits[i1].resid.^2);
end

# Find the best fit
phi1_orig = exp(all_fits[findmin(all_costs)[2]].param);


# Compute the Hessian around best fit
x_loc = hcat(x_grid...);
model_cur(phi::AbstractArray, x::AbstractArray) = BK_simulator(phi, x, model_id=0)[:]
data_vec = ydata_orig;
f_cost = phi_var -> sum((model_cur(phi_var, x_loc)[:].-data_vec).^2); # Current cost function (quadratic)
g_cost = phi_var -> log_deriv_wrapper(f_cost, phi_var); # Define which parameters we're taking in log space ([default: all])
f_res = phi_var -> (model_cur(phi_var, x_loc)[:].-data_vec) # Return residuals
g_res = phi_var -> log_deriv_wrapper(f_res, phi_var, log_specific=log_specific);

# Determine the hyperellipsoid around our best fit:
hess_orig = ForwardDiff.hessian(g_cost, log(phi1_orig)); #log hessian
D_orig, V_orig = eig(hess_orig)

([64.35037264881387,13.830148966622373,0.6559551497371164,0.18417172630039122,0.08307059310262276,0.004508785833056124,3.793040737186234e-6,0.0004923648254559906],
8x8 Array{Float64,2}:
 -0.153574   -0.00928801   0.139115   -0.0472614  …  -0.969698     0.0620004 
 -0.0459658  -0.0990201    0.0540569  -0.208893       0.00231223  -0.908613  
 -0.407168    0.108185     0.24282     0.730567       0.0113598   -0.283012  
 -0.399084   -0.79809     -0.371863   -0.0625434     -0.00400545   0.0188488 
  0.186752   -0.131102     0.589223   -0.355179      -0.003727    -0.0996216 
 -0.394937    0.448513    -0.322761   -0.485836   …  -0.00256519  -0.0971814 
 -0.605651   -0.0438131    0.536328   -0.230815       0.243898     0.265966  
 -0.302629    0.348118    -0.203655    0.032033      -0.00532997  -0.00446693)

In [4]:
phi1_orig

8-element Array{Float64,1}:
   3.2117e-8 
   0.162934  
   0.0607788 
   0.636445  
   5.28787e-5
   2.53238   
 135.554     
   7.99891   

## Fit noisy log assay and compute Hessian around best fit

In [8]:
# Store multiple fits (different initial point for LM, takes around 5 sec / fit)
all_fits = Array(Any, 24)
all_costs = Array(Any, length(all_fits))

for i1 = 1:length(all_fits)
    if i1==1
        phi_init = phi0;
    else
        phi_init = phi0.*(1+(2*rand(size(phi0))-1)*0.9) # Add up to 90% noise to initial params
    end
    all_fits[i1] = curve_fit((x,p) -> log(abs(BK_simulator(exp(p), x))), hcat(x_grid...), ydata_log, log(phi_init)); # abs needed due to weird julia error, BK_simulator has an output range between 0 and 1
    all_costs[i1] = sum(all_fits[i1].resid.^2);
end

# Find the best fit
phi1_log = exp(all_fits[findmin(all_costs)[2]].param);

# Compute the Hessian around best fit
x_loc = hcat(x_grid...);
model_cur(phi::AbstractArray, x::AbstractArray) = log(BK_simulator(phi, x, model_id=0)[:])
data_vec = ydata_log;
f_cost = phi_var -> sum((model_cur(phi_var, x_loc)[:].-data_vec).^2); # Current cost function (quadratic)
g_cost = phi_var -> log_deriv_wrapper(f_cost, phi_var); # Define which parameters we're taking in log space ([default: all])
f_res = phi_var -> (model_cur(phi_var, x_loc)[:].-data_vec) # Return residuals
g_res = phi_var -> log_deriv_wrapper(f_res, phi_var, log_specific=log_specific);

# Determine the hyperellipsoid around our best fit:
hess_log = ForwardDiff.hessian(g_cost, log(phi1_log)); #log hessian of log assay
D_log, V_log = eig(hess_log)

5.198712624863577 (predicted_residual) >
5.198712624863575 (residual) + 8.881784197001252e-16 (eps)
5.19871262486358 (predicted_residual) >
5.198712624863577 (residual) + 8.881784197001252e-16 (eps)


([2158.479471687392,593.8734618560258,142.87479090644592,17.701088928327998,14.463313928239026,0.0031558952478074275,1.631646399566662,3.174442385790068],
8x8 Array{Float64,2}:
  0.195796   -0.0555258  -0.160136   …   0.728488     0.511638    0.080888 
 -0.0777834  -0.180006    0.0919183      0.505161    -0.551803    0.548353 
  0.439421   -0.276803   -0.327186       0.215543    -0.303473   -0.635142 
 -0.321543   -0.80811     0.414321      -0.00389179   0.0915457  -0.198386 
 -0.248346   -0.022202   -0.201808       0.0014504    0.100292   -0.0410389
  0.515843    0.160524    0.612106   …   0.00649231  -0.262223   -0.0648761
  0.491457   -0.456366   -0.342305      -0.409372     0.0605761   0.486103 
  0.299722   -0.0179272   0.39322       -0.00369459   0.500774    0.0883991)

In [9]:
#save results
save("saves/figure_A1_hess"*cur_time*".jld", "hess_orig", hess_orig, "D_orig", D_orig,
"hess_log", hess_log, "D_log", D_log, "phi1_orig", phi1_orig, "phi1_log", phi1_log, "ydata_orig", ydata_orig, "ydata_log", ydata_log)

In [10]:
[log(sort(abs(D_orig))) log(sort(abs(D_log)))]

8x2 Array{Float64,2}:
 -12.4823    -5.75848
  -7.61629    0.48959
  -5.40173    1.15513
  -2.48806    2.67162
  -1.69189    2.87363
  -0.421663   4.96197
   2.62685    6.38667
   4.16434    7.67716

### Do the full reduction

include("../BK_functions/bk_setup_script.jl")


# Run MBAM from those parameters
cur_time = timestamp();

#outp = load("saves/figure_A1_20161108T15185000.jld", "outp") #Step 1
outp = load("saves/figure_A1_20161108T17280800.jld", "outp") #Step1-2


# First iteration:
outp = MBAM(BK_simulator, phi1, x_grid, model_iters=[70, 71, 72], 
move_dir=[-1, -1, -1], boundary_time=10, verbose=1, reduce_func=BK_reduce, data_vec=ydata, prevRunResults=outp)

using JLD
save("saves/figure_A1"*cur_time*".jld", "outp", outp, "phi1", phi1, "ydata", ydata)

# Run MBAM from those parameters

# First iteration:
outp_log10 = MBAM(BK_simulator, phi1, x_grid, model_iters=[51], 
move_dir=[-1], boundary_time=10, verbose=1, reduce_func=BK_reduce, data_vec=log10(ydata))

using JLD
save("saves/figure_A1_log"*cur_time*".jld", "outp_log10", outp_log10, "phi1", phi1, "ydata", ydata)